<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 运行时函数

In [ ]:
from videos.walkthroughs import walkthrough_22 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习如何将自定义函数转换为可包含在 LangChain 链中的运行时。

---

## 目标

完成这个 notebook 后，您将能够：

- 理解如何创建自定义的运行时函数并将其包含在您的 LangChain 链中。
- 使用自定义运行时函数在将数据发送到 LLM 之前进行预处理。
- 使用自定义函数批量翻译原始文本为提示模板。
- 创建一个利用多个自定义运行时函数的 LangChain 情感分析链。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 使用 `RunnableLambda` 创建自定义运行时函数

我们已经看到 LangChain 提供的可组合运行时，包括 LLM 实例、提示模板和输出解析器。LangChain 另一个强大的工具是能够通过 `RunnableLambda` 将任意函数转换为运行时函数。

In [ ]:
from langchain_core.runnables import RunnableLambda

我们将从一个简单的数学函数开始探索自定义运行时函数。

In [ ]:
def double(x):
    return 2*x

这段简单的 Python 函数没有 LangChain 可运行的 `invoke`（或 `batch` 或 `stream`）方法，这并不令人惊讶。

In [ ]:
try:
    double.invoke(2)
except AttributeError:
    print('`double` is a Python function and does not have an `invoke` method.')

然而我们可以通过将其传递给 `RunnableLambda` 来轻松地将其转换为 LangChain 运行时函数。

In [ ]:
runnable_double = RunnableLambda(double)

In [ ]:
runnable_double.invoke(6)

In [ ]:
runnable_double.batch([2, 4, 6, 8])

和其它运行时一样，`runnable_double` 这样的自定义函数运行时可以组合成链。

In [ ]:
multiply_by_eight = runnable_double | runnable_double | runnable_double

In [ ]:
multiply_by_eight.invoke(11)

您可以尽情发挥创造力在链中利用自定义函数，本 notebook 的后续部分，我们将探讨几个自定义运行时函数在链中的常见用例。

---

## 数据管理

无论是格式化、校正还是验证，您可能希望在与 LLM 交互之前或之后，对经过链的数据进行处理。 

举个例子，假设您正在构建一个情感分析应用，分析用户评论的情感。用户评论可能包含各种不一致的地方，比如大小写混用、包含多余的空格和缩写。在将文本发送到 LLM 之前对其进行规范化，可以提高情感分析的准确性。

以下的 `normalize_text` 函数将通过转换为小写、扩展缩写（expanding contractions）和移除多余空格来规范文本。

In [ ]:
import re
import contractions # pip install contractions

def normalize_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Expand contractions
    text = contractions.fix(text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

---

## 练习：创建规范化文本的运行时函数

利用您目前为止学到的关于创建运行时函数的知识，从上面提供的 `normalize_text` 函数创建一个运行时函数。

成功实现后，您应该能用它批量处理以下示例评论列表。

如果遇到困难，可以查看下面的*参考答案*。

In [ ]:
reviews = [
    "I LOVE this product! It's absolutely amazing.   ",
    "Not bad, but could be better. I've seen worse.",
    "Terrible experience... I'm never buying again!!",
    "Pretty good, isn't it? Will buy again!",
    "Excellent value for the money!!! Highly recommend."
]

### 您的代码

### 参考答案

In [ ]:
RunnableLambda(normalize_text).batch(reviews)

---

## 为提示词模板格式化文本

在上一个练习中，您得到了一个规范化的评论列表，如下所示。

In [ ]:
normalized_reviews = [
    'i love this product! it is absolutely amazing.',
    'not bad, but could be better. i have seen worse.',
    'terrible experience... i am never buying again!!',
    'pretty good, is not it? will buy again!',
    'excellent value for the money!!! highly recommend.'
]

假设我们现在想把这些规范化的评论传入一个用于情感分析的提示模板，比如下面的 `sentiment_template`。

In [ ]:
sentiment_template = ChatPromptTemplate.from_template("""In a single word, either 'positive' or 'negative', \
provide the overall sentiment of the following piece of text: {text}""")

在之前的 notebook 中我们了解到，要调用上面的模板，需要传入一个包含占位符键（上面模板中的 `{text}`）的字典，例如：

In [ ]:
sentiment_template.invoke({"text": 'i love this product! it is absolutely amazing.'})

因此，为了准备 `normalized_review` 中的项目以便传入 `sentiment_template`，我们需要把每一行文本转换为一个字典，键（key）为 `"text"`，值（value）为实际的文本行。

让我们创建一个运行时 lambda 来实现这个功能。由于工作量非常小，就直接使用 lambda 函数。

In [ ]:
prep_for_sentiment_template = RunnableLambda(lambda text: {"text": text})

现在我们可以使用 `prep_for_sentiment_template` 来准备 `normalized_reviews` 以便用于 `sentiment_template`。

In [ ]:
prep_for_sentiment_template.batch(normalized_reviews)

---

## 练习：创建情感分析链

在这个练习中，创建一个情感分析链，您可以将上面的原始 `reviews` 列表批量传入。

您的链应：
- 规范化原始评论。
- 准备规范化的评论以便在 `sentiment_template` 中使用（如上所定义）。
- 将准备好的规范化评论通过 `sentiment_template` 传递。
- 将提示词模板传递给 `llm`（如上已定义）。
- 最后用 `StrOutputParser` 的实例解析 LLM 输出，您需要实例化它。

如果遇到困难，可以查看下面的参考答案。

### 您的代码

### 参考答案

这个链唯一还没创建的组件是输出解析器，所以我们在这里创建它。

In [ ]:
parser = StrOutputParser()

创建好所有的运行时后，就可以组合这个链了。

In [ ]:
sentiment_chain = RunnableLambda(normalize_text) | prep_for_sentiment_template | sentiment_template | llm | parser

现在就可以通过链批量处理原始评论了。

In [ ]:
sentiment_chain.batch(reviews)

---

## 总结

在这个 notebook 中，您学习了如何创建自定义运行时以便包含在您的链中。实际上，链本身就是运行时，下一个 notebook 中，您将开始学习如何将链串联在一起。